<a href="https://colab.research.google.com/github/nicolasvazquez95/nanofoams_utils/blob/main/scripts/build_nanofoams_tensorflow_colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input files and scripts 😀

## Input file 

In [105]:
%%writefile 150a0_161.txt
###############################
# Input script nanofoams_input.txt
# For using with script build_nanofoams.py (CPU Parallel version) or with the TensorFlow version
# Nota: The units of measure can be chosen by user (use always the same units!)
###############################

# Lattice type (Cubic:0,BCC:1,FCC:2) (if the value is different of 1 or 2, lattice is cubic by default)
1

# Box size: x_size y_size z_size (Units AA)
495.45
495.45
495.45

# Lattice parameter
3.303

# Solid fraction of foams
0.30

# F data
# Mean size ligament diamenter (this value can be a little different when the foam is generated, the program informs the user of the changes)
22.2177
# Number of waves
15000

# a constant 
495.45

# Random seed
462022

Overwriting 150a0_161.txt


## H_generator.py (generator of directions)

In [106]:
%%writefile H_generator.py

#############################
# Directions generator from a given value H
### How to use? Type in command line
    ## python H_generator.py H2value1 H2value2 ... H2valuen
# Se crearán entonces archivos para cada valor de H separado, con las direcciones generadas.
# For each given value of H, a file will be created, with the directions generated.
# Last modification: 18/02/22
## We've added a function makeH2database for databases. 
#   from H_generator import makeH2database
#   makeH2database(H2max,threshold,filename) 
#############################

# Modules
import sys
import numpy as np
import itertools
import random

def H2_shortlist(H2):
    '''
    It returns a list of Miller indices (h,k,l), which are solutions of the equation
    H^2=h^2+k^2+l^2. Also, the values satisfy h>k>l.
    Input: H2 (H^2 value)
    '''
    HKL = []
    for h in range(H2):
        for k in range(h,int(np.sqrt(H2)+1)):
            for l in range(k,int(np.sqrt(H2)+1)):
                if (h**2+k**2+l**2==H2):
                    hkl = [h,k,l]
                    HKL.append(hkl)
    return HKL

def HKL_sign(hkl):
    '''
    It returns all possible permutations of (h,k,l) changing signs (+-)
    Input: shortlist [h,k,l]
    '''
    combinations = []
    if hkl.count(0)== 0: # Caso sin ceros
    # Pick up two random elements
        options = [(0,1),(0,2),(1,2)]
        choice = random.choice(options)
        for sign_1 in (1,-1):
            my_hkl = list(hkl)
            my_hkl[choice[0]] = sign_1*my_hkl[choice[0]]
            for sign_2 in (1,-1):
                my_hkl[choice[1]] = sign_2*my_hkl[choice[1]]
                combinations.append(np.array(my_hkl))
        return combinations
    elif hkl.count(0)==1: # 1 zero case
        index_0 = hkl.index(0)
        my_hkl = list(hkl)
        options = [0,1,2]
        del options[index_0]
        choice = random.choice(options)
        for sign in (1,-1):
            my_hkl[choice] = sign*my_hkl[choice]
            combinations.append(np.array(my_hkl))
        return np.array(combinations)
    else: # 2 zeros
        return [np.array(hkl)]

def total_directions(H2,unpack=False):
    '''
    Ir returns a list of lists, with every possible combination of vectors, all of them build from a given H^2 value.
    Input: H2 (valor de H^2).If unpack=True, it returns a single list with all directions.
    '''
    directions = []
    shortlists = H2_shortlist(H2)
    
    for shortlist in shortlists:
        if shortlist.count(0)<2:
            permutations = list(itertools.permutations(shortlist))
            # Delete equivalent elements
            permutations = list(dict.fromkeys(permutations))
        else:
            permutations = []
            h,k,l = shortlist
            permutations.append([h,k,l]);permutations.append([l,h,k]);permutations.append([k,l,h])
            
        for permutation in permutations:
            if unpack==False:
                directions.append(HKL_sign(permutation))
            else:
                vectors = HKL_sign(permutation)
                for vector in vectors:
                    directions.append(vector)
    return directions

def writeH2_file(H2,filename):
    """
    It saves a txt file with the directions generated.
    Input: H2,filename
    """
    directions = total_directions(H2,unpack=True)
    header = 'H2 value: '+str(H2)+'\n'
    header2 = 'Number of directions: '+ str(len(directions))
    np.savetxt(filename,directions,fmt="%d",header=header+header2)

def makeH2database(H2max,threshold,filename='H2_database.txt'):
    """
    It creates a file 'filename.txt' from a H^2 value y and number of possible directions, only if N_Dir>=threshold,
    from 0 to H2max.
    """
    Dir = []
    N = []
    for i in range(H2max):
        if total_directions(i)==None:
            Dir.append(0)
        else:
            Dir.append(len(total_directions(i,unpack=True)))
        N.append(i)
        print('{}/{} completed.'.format(i+1,H2max),end='\r',flush=True)
    Dir = np.array(Dir,dtype=np.int32)
    N = np.array(N,dtype=np.int32)
    
    Dir_N = np.column_stack((N,Dir))
    Dir_N_filtrada = Dir_N[Dir_N[:,1]>=threshold]
    np.savetxt(filename,Dir_N_filtrada,fmt='%d',header='H2 Database - Max value {}, Min Directions {}'.format(H2max,threshold))
# main
if __name__ == "__main__":
    for i in range(1,len(sys.argv)):
        H2 = sys.argv[i]
        filename = 'H2_'+H2+'.txt'
        writeH2_file(int(H2),filename)

Overwriting H_generator.py


# The program

## Imports

In [107]:
import tensorflow as tf
import numpy as np
import scipy as sp
from scipy import special
from os.path import exists as file_exists

## Read input script

In [108]:
input_script = '150a0_161.txt'
struct_info = np.loadtxt(input_script,dtype=np.float32)

lattice = int(struct_info[0])

x_size = struct_info[1]
y_size = struct_info[2]
z_size = struct_info[3]

lattice_parameter = struct_info[4]

phi_b = struct_info[5]
if not (0<=phi_b<=1):
    raise Exception('phi_b must be a number between 0 and 1.')

## With phi_b we can calculate xi, as in Soyarslan paper
xi = np.sqrt(2)* sp.special.erfinv(2*phi_b-1)

# Number of waves
N = int(struct_info[7])
# a constant
a = struct_info[8]

# Reduced box sizes
x_size_reduced = int(x_size / lattice_parameter)
y_size_reduced = int(y_size / lattice_parameter)
z_size_reduced = int(z_size / lattice_parameter)

In [109]:
# H_generator import and run
import H_generator

if not file_exists('H2_database.txt'):
    H_generator.makeH2database(500, 48)
    print('Warning: H2_database.txt not found. New database file created instead.')
else:
    print('Using existing directions file '+'H2_database.txt')

H2_database = np.loadtxt('H2_database.txt',dtype=np.int32)
L_mean = struct_info[6]
a = struct_info[8]
H2_real = ((a/L_mean)*(0.53*phi_b+0.41))**2

print('H^2 value calculated:',H2_real)
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
    
H2_final = find_nearest(H2_database[:,0],H2_real)
    
print('Nearest value of H^2 in database:',H2_final)
    
L_mean_new = (a/np.sqrt(H2_final))*(0.53*phi_b+0.41)
print('New L_mean:',L_mean_new)
print('Relative error in mean ligament diameter:',np.abs(1-(L_mean/L_mean_new)),'\n')

filename_H2 = 'H2_'+str(H2_final)+'.txt'

if not file_exists(filename_H2):
    H_generator.writeH2_file(H2_final, filename_H2)
    print('Directions file created.\n')
else:
    print('Using existing directions file '+filename_H2)

# Directions from txt
print("Loading directions from txt...",end='',flush=True)
directions = tf.constant(np.loadtxt(filename_H2,dtype=np.float32))
N_dir = len(directions)

pi2a = 2*np.pi/a
q0 = pi2a * np.sqrt(H2_final)
Qi = pi2a * directions
print(" Ready.\n")
# Set seed
random_gen = tf.random.Generator.from_seed(int(struct_info[9]))
tf.random.set_seed(int(struct_info[9]))

print("Initializing phases and vectors...")
# Inicializamos array de fases
Phi = np.pi*(-1 + 2*random_gen.uniform(shape=(N_dir,int(N/N_dir))))
print('Phi shape:',Phi.shape)
Qi = pi2a * directions
print('Number of waves, Number of directions:',(Phi.shape[0]*Phi.shape[1],N_dir))
print("Mean phi,Std phi: ",(tf.math.reduce_mean(Phi).numpy(),tf.math.reduce_std(Phi).numpy()))
print("Mean wave direction:",(1/N_dir)*tf.math.reduce_sum(Qi,axis=0).numpy(),'\n')

Using existing directions file H2_database.txt
H^2 value calculated: 161.00007068500597
Nearest value of H^2 in database: 161
New L_mean: 22.217704007987045
Relative error in mean ligament diameter: 2.231141310593543e-07 

Using existing directions file H2_161.txt
Loading directions from txt... Ready.

Initializing phases and vectors...
Phi shape: (96, 156)
Number of waves, Number of directions: (14976, 96)
Mean phi,Std phi:  (0.020793345, 1.835302)
Mean wave direction: [0.02642036 0.02430674 0.03804532] 



## Meshgrid creation

In [110]:
print("Creating meshgrid...")
x,y,z = tf.meshgrid(range(x_size_reduced),range(y_size_reduced),range(z_size_reduced),indexing='ij')
x = lattice_parameter*tf.cast(x,tf.float32)
y = lattice_parameter*tf.cast(y,tf.float32)
z = lattice_parameter*tf.cast(z,tf.float32)

_x = tf.reshape(x, (-1,1))
_y = tf.reshape(y, (-1,1))
_z = tf.reshape(z, (-1,1))

points = tf.squeeze(tf.stack([_x, _y,_z], axis=-1))

if lattice == 1: 
    print('Lattice type: BCC')
    x_bcc = x + lattice_parameter/2
    y_bcc = y + lattice_parameter/2
    z_bcc = z + lattice_parameter/2
    
    _x_bcc = tf.reshape(x_bcc, (-1,1))
    _y_bcc = tf.reshape(y_bcc, (-1,1))
    _z_bcc = tf.reshape(z_bcc, (-1,1))
    
    points_bcc = tf.squeeze(tf.stack([_x_bcc, _y_bcc,_z_bcc], axis=-1))
    
    points = tf.concat([points,points_bcc],axis=0)

    del x_bcc,y_bcc,z_bcc
elif lattice==2:
    print('Lattice type: FCC')
    x_fcc = x + lattice_parameter/2
    y_fcc = y + lattice_parameter/2
    z_fcc = z + lattice_parameter/2
    
    _x_fcc = tf.reshape(x_fcc, (-1,1))
    _y_fcc = tf.reshape(y_fcc, (-1,1))
    _z_fcc = tf.reshape(z_fcc, (-1,1))
    
    points_fcc_xy = tf.squeeze(tf.stack((_x_fcc, _y_fcc, _z), axis=-1))
    points_fcc_xz = tf.squeeze(tf.stack((_x_fcc, _y, _z_fcc), axis=-1))
    points_fcc_yz = tf.squeeze(tf.stack((_x, _y_fcc, _z_fcc), axis=-1))
    points = tf.concat((points,points_fcc_xy,points_fcc_xz,points_fcc_yz),axis=0)

    del x_fcc,y_fcc,z_fcc
else:
    print('Lattice type: Cubic')
    pass

# Delete meshgrid 
del x,y,z 

NParticles = points.shape[0]
print("Ready. Number of atoms: "+str(NParticles),'\n')

Creating meshgrid...
Lattice type: BCC
Ready. Number of atoms: 6750000 



## $f$ evaluation in mesh grid

In [111]:
print("Evaluating f in lattice...")
sqr2N = np.sqrt(2/N).astype(np.float32)
# Batches allocation
batch_size=32 # Hyperparameter
print("Batch size:",batch_size,'\nAllocating batches...')
# A function for the batches generation
def make_batches(NParticles=NParticles,batch_size=batch_size):
    if NParticles%batch_size==0:
        return int(NParticles/batch_size)
    else:
        splits = []
        _NParticles = NParticles
        while _NParticles>batch_size:
            splits.append(batch_size)
            _NParticles-=batch_size
        splits.append(NParticles%batch_size)
        return tf.constant(splits)
point_batches = tf.split(points,num_or_size_splits=make_batches(),axis=0)
print("Ready. Number of batches:",len(point_batches))
# Define a function which evaluates f for one batch
@tf.function
def chunk_f(batch):
    grid_dot = tf.tensordot(Qi,tf.transpose(batch),axes=1)
    terms = tf.cos(tf.transpose([grid_dot]) + Phi)
    return sqr2N*tf.math.reduce_sum(terms,axis=(1,2))

Evaluating f in lattice...
Batch size: 32 
Allocating batches...
Ready. Number of batches: 210938


In [ ]:
# Iterate over the whole set
progbar = tf.keras.utils.Progbar(len(point_batches))
f = []
for n,batch in enumerate(point_batches):
  f.append(chunk_f(batch))
  progbar.update(n)
# Classify particles
f = tf.concat(f,axis=0)
@tf.function
def type_particle(f):
    if f>xi or  tf.experimental.numpy.isclose(f,xi):
        return 2 # Void
    else: return 1 # Solid
types = tf.vectorized_map(type_particle,f)
print("\nEvaluation completed.")

 19565/210938 [=>............................] - ETA: 2:19

Exception ignored in: <function ScopedTFGraph.__del__ at 0x7fb4697f5d40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/c_api_util.py", line 53, in __del__
    def __del__(self):
KeyboardInterrupt


 58751/210938 [=======>......................] - ETA: 1:45

## Save data file

In [96]:
print("Saving data file...")
# OVITO preamble
timestep = "ITEM: TIMESTEP \n0\n"
NAtoms = "ITEM: NUMBER OF ATOMS \n"+str(NParticles)+'\n'
bounds = "ITEM: BOX BOUNDS pp pp pp \n"

size_x = '0.0 '+str(x_size)+'\n'
size_y = '0.0 '+str(y_size)+'\n'
size_z = '0.0 '+str(z_size)+'\n'
dimensions = size_x+size_y+size_z
properties = "ITEM: ATOMS id type x y z f"

preamble = timestep+NAtoms+bounds+dimensions+properties
# Build a matrix for the whole set of data
ID = [i+1 for i in range(NParticles)]
data = np.column_stack([ID,types.numpy(),points[:,0].numpy(),points[:,1].numpy(),points[:,2].numpy(),f.numpy()])
# Save file
np.savetxt('data_'+input_script[:-4]+'.txt',data,fmt='%d %d %f %f %f %f',header=preamble,comments='')
print("Data file saved succesfully.\nExiting program...")

Saving data file...
Data file saved succesfully.
Exiting program...


# Get dump file

In [ ]:
# It only download in Google Chrome
from google.colab import files
files.download('data_'+input_script[:-4]+'.txt') 

# Google Drive connection! It's faster than Colab. Copy files there and download from Drive.
from google.colab import drive
drive.mount('/content/drive')